## Summary

This package allows to keep track of activities, gains, and losses of a
investment portfolio. Use commands to log purchase and sales of shares,
dividends, and to update the value of the portfolio.

The basic feature of the package is that it can produce tables containing
the relative value of each share. This takes into account the purchase date,
purchase price and fee, dividends that were paid, the current value of the
shares, the sales fee, and the number of days for which the investment was
held. A relative value of p means that the investment is equivalent to having
put the purchase amount into a savings account with continously compounded
interest of p percent per year.

A number of additional tools are provided, e.g. a method that computes bond
prices. The significance of p values can be understood by analysing historic
data. The notebook <code>ta_work</code> can be used for working on the
account -- it contains a list of all callable functions (for which the
docstrings can be displayed).

## Initialisation

To demonstrate the trading account (TA) logbook package,
we first execute the master file. This will load all necessary
python packages and all the TA classes and methods.

In [ ]:
# execute master script
master_file = open("./ta_master.py")
exec(master_file.read())
master_file.close()

If no files containing TA data etc. are found, new ones will be
initiated. Several TA logbooks can be kept in parallel. We can
check which ones are present by running the following command.

In [ ]:
account_name()

If the working directory was clean, the produced list will only
contain the default account <code>ta</code>. We can add a new one by
passing its name to the function <code>account_name()</code>. We now create
a demo account for this demonstration. If <code>demo</code> is already
present, the command will simply switch to it.

In [ ]:
account_name('demo')

In this list of accounts, the first entry is always the active
account. The functionalities of the TA package will be
demonstrated in this notebook, but we can always have the
docstrings displayed for further details, e.g.:

In [ ]:
print(account_name.__doc__)

The callable methods are all listed in the <code>ta_work</code> notebook.
The logbooks for each files are pandas dataframes, that are saved
in the working directory. Once the software is initialised, we
can simply run the commands described below to act on the active
TA (e.g., use <code>ta_work</code>) for that. Before doing that, we
reset the <code>demo</code> TA:

In [ ]:
if os.path.isfile('demo_save.p'):
    os.remove('demo_save.p')

## Logging TA Activities

We first deposit 5000 in the account, then add and withdraw, and make a small adjustment.

In [ ]:
account_activity(5000, date=pd.Timestamp(2017, 1, 1))
account_activity(1000, date=pd.Timestamp(2017, 2, 1))
account_activity(-500, date=pd.Timestamp(2017, 3, 1))
account_activity(5, comment='Adjustment', date=pd.Timestamp(2017, 4, 1))

A new TA file with the name <code>demo</code> with an opening deposit of 5000 was created. If <code>date</code> is not given, the current date will be used -- this is what one would do in practice (in this demo, dates are prescribed to demonstrate the changes in the account over a period of a few months). We can now display the account:

In [ ]:
all_values()

The different display options will be discussed in the next section. Comments are set automaticall, but the default comments can be overwritten, cf. the docstring for <code>account_activity()</code>.  First, let us by some shares, namely 2000 worth of <code>Share1</code>, with an order fee of 10.

In [ ]:
buy('Share1', 2000, 10, date=pd.Timestamp(2017, 5, 1))
all_values()

The value of the shares plus the order fee is automatically subtracted from the account balance. Before interpreting the displayed values, let us update the share values: suppose that after 1 month, the value of the share has increased by 50:

In [ ]:
update(Share1=2050, date=pd.Timestamp(2017, 6, 1))
all_values()

The first argument in parentheses is the share value. The second value is the total amount of dividends paid for this share -- 0 so far. The value in front of the parentheses, 0.1744, is the relative share value -- having this information in the TA logbook was the main motivation for writing this package. It gives the continously compounded interest rate necessary to make the same profit over the given time period:

costs, on 05-01:  2010,

payout, on 06-01:  2040   (the share value minus an estimated order fee of 10),

and to turn 2010 into 2040 within one month, we would need a savings account with a continuously compounded interest rate of 17.44%:

In [ ]:
2010 * math.exp( float(31)/365 * 0.1744)

Now, whether or not this is high enough to encourage us to sell the share is a different story -- it depends on the properties of the share, the market, and our expectations. Tools to gauge the significance of relative share values such as 0.1744 are included in the package and will be discussed later. Had there been any dividend payments for the share, then they would be taken into account for the relative value. Next, buy another share and update share values:

In [ ]:
update(Share1=2050, date=pd.Timestamp(2017, 7, 1))
buy('Share2', 1000, 10, date=pd.Timestamp(2017, 7, 1))
update(Share1=2040, Share2=1020, date=pd.Timestamp(2017, 8, 1))
all_values()

For <code>update()</code>, the current share values are given as a dictionary with the share names as keys. If those values are not given, only the relative share values will be updated according to their time-dependancy. Note how the relative value of <code>Share1</code> has decreased in row 6 despite the share value remaining the same -- that's because more time has passed. Next we log a dividend payment and then sell one of the shares.

In [ ]:
dividend('Share1', 200, date=pd.Timestamp(2017, 9, 1))
update(Share1=2070, Share2=1050, date=pd.Timestamp(2017, 10, 1))
all_values()

In [ ]:
sell('Share1',2065,date = pd.Timestamp(2017,10,1))
update(Share2=1080,date = pd.Timestamp(2017,11,1))
all_values()

When selling, one would simply pass the total amount credited to the account as an argument. E.g., here, the share value of 2070 minus a 5 order fee. Upon selling, a message is produced stating the overall relative return (here, this return is a little bit better than the last relative value that was displayed, because the order fee for the sale turned out to be less than the 10 that are used for computing the relative values; the default sales fee can be set in the code). Note that the values for <code>Share1</code> are not displayed, because it is not an active share anymore, cf. next section. If a mistake is made handling the account, it can be undone as follows.

In [ ]:
account_activity(-3, comment='Mistake', date=pd.Timestamp(2017, 11, 1))
all_values()

In [ ]:
delete_last_row()
all_values()

Other corrections have to be done manually. The correction method <code>delete_last_row()</code> also calls the function <code>backup()</code>, which should be run from time to time. It saves the dataframe, both as a python object and as a spreadheet (with a timestamp). 

## Dislaying the TA Log

In the displayed TA dataframes above, the column for <code>Share1</code> was dropped after it had been sold. It is still in the dataframe, of course, and it can be displayed as follows.

In [ ]:
all_values(all_shares=True)

Now, the entries in the columns with share values are strings. We can also generate dataframes with only the share values or the relative values as floats.

In [ ]:
df = shr_values(all_shares=True)
df

Use <code>rel_values()</code> to produce a dataframe showing the relative share values. Note that the data was reindexed over time, so that it can be plotted.

In [ ]:
df.plot()

The three different display methods <code>all_values(), rel_values(), shr_values()</code> have different default formats, but this can be overwritten with keyword arguments, e.g. <code>shr_values(comments=True, date_as_index=False)</code>. Cf. the docstrings, e.g.:

In [ ]:
print(shr_values.__doc__)

## Other Methods on the Dataframe

List the shares in the portfolio using <code>active_shares()</code> or <code>all_shares()</code>. The methods <code>delete_last_row()</code> and <code>backup()</code> have already been discussed. Lastly, the total value of the portfolio:

In [ ]:
total_value()

## Investment tools

Suppose we are considering to buy a certain bond with a coupon (yield) of 4.3% and 15 years to maturity. Then we can use the following function to get an overview of bond prices and the corresponding overall return.

In [ ]:
bond_evaluation(4.3, 15)

This means that, if we want a return of at least 5%, we should not pay more than 92.73. This is an unrelated functionality -- the following methods are more geared towards the main features of the package described above.

The following methods help gauge the significance of relative values p -- this helps to plan ahead and also to decide whether to hold or to sell a share. For example, if a share with very low volatility has a p-value double its average historical rate of return, then this is a exceptional event, and we should sell before the share price reverts to the mean. For more volatile shares, such a p-value is not as significant and we may consider holding.

To do this, download weekly closing prices and save it in the working directory as <code>data.csv</code>. Then run the following to find the average weekly growth and its standard deviation from that mean (alternatively, a list of weekly closing prices can be passed).

In [ ]:
parameters = find_mu_sigma()
parameters

Next, we use this data for a Monte-Carlo simulation of shares with those parameter. This produces a distribution of maximum achieved relative rates.

In [ ]:
simulate_p(parameters[0], parameters[1])

The numbers in the second row state the weeks in which those p-values were achieved (only p values between certain weeks are considered, cf. the docstring for <code>simulate_p()</code>).

Let's check that a lower volatility gives a more concentrated distribution of p-values.

In [ ]:
simulate_p(parameters[0], 0.001*parameters[1], name='demoshare')

This time, the name of the share was given as an keyword argument. In this case, the above data is written in a spreadsheet in the working directory. One can later consult this spreadsheet when making decisions.

Lastly, as a sanity check, let us make sure that the median p found in the low-volatility simulation really corresponds to the average weekly rate found above:

In [ ]:
p_med = 0.2308 # = p_50 in the previous table
               # change if you were running this on different data
               # should approximately give parameters[0]
math.exp(p_med)**(float(1)/52)

## Web Scraping Functionalities

Suppose there are stocks we'd like to buy, but they are currently overpriced. We can keep a list of stocks, <code>watchlist.txt</code>, with the Yahoo Finance handles and the stock prices at which we would buy. Calling the following function will check the watchlist (this is a separate script, so, it can be run from outside the whole ta package). This is very helpful once that list gets very long.

In [ ]:
check_watchlist()

NEXT, INTRODUCE AUTO-UPDATE

## Getting Started

Now that you know how everything works, just run

<code>account_name('your_account_name')</code>

and start managing your account! This can be done in another notebook (e.g. <code>ta_work</code>) or from the python command line -- just execute <code>ta_master.py</code> first to load everything.

## To-Do

Other functionalities that could be implemented: a method that scrapes the values of active shares off the internet, so that one does not have to look up and enter the share values for <code>update()</code> manually; similarly, an automation of finding the significance levels using scraped data.

Please email me with any questions, concerns, suggestions.